In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, models
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [3]:
set_size = x_train.shape[0]
downsampling_ratio = 10
#####
sample_no = np.arange(set_size)
sample_no = np.random.choice(sample_no, int(set_size/downsampling_ratio), replace=False)
x_train = x_train[sample_no]
y_train = y_train[sample_no]
##
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
##
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)


In [4]:
# (x_train, y_train)[:].shape

In [5]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [6]:
def build(input_shape, no_classes):
    model = models.Sequential()
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.3))
    ##
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.3))
    ##
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.3))
    ##
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.3))
    ##
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(no_classes, activation='softmax'))
    ##
    return model

In [7]:
model = build(x_train.shape[1:], 10)

In [8]:
# model.summary()

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
convergence_accuracy_diff = 0.002
converge = False
old_acc = 0
num_epoch_to_converge = 0
while converge==False:
  num_epoch_to_converge += 1
  hist = model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=1, validation_data=(x_test, y_test))
  new_acc = hist.history['accuracy'][-1]
  if (new_acc - old_acc) < convergence_accuracy_diff:
    converge = True
  old_acc = new_acc

Epoch 1/2
79/79 [==============================] - 82s 1s/step - loss: 1.7107 - accuracy: 0.3830 - val_loss: 2.9750 - val_accuracy: 0.1631
Epoch 2/2
79/79 [==============================] - 81s 1s/step - loss: 1.5630 - accuracy: 0.4340 - val_loss: 2.9835 - val_accuracy: 0.1777
[0.382999986410141, 0.4339999854564667]
0.4339999854564667


In [ ]:
    #train error
    _, train_accuracy = model.evaluate(x_train, y_train)
    train_error = (1 - train_accuracy)*100
#     y_train_error.append(train_error)
    #test_error
    _, test_accuracy = model.evaluate(x_test, y_test)
    test_error = (1 - test_accuracy)*100
#     y_test_error.append(test_error)
print('train error = {}\ntest error = {}'.format(train_error,test_error))